# Estimating two dimensional state (position and velocity) with single dimension observation

Now that we have some experience with multi-variate normals, let's expand our use of the Kalman filter from 1D to 2D. This example will be very similar to the previous example, but now we will use a 2D state model:

$x_{t+1} = x_t + dt \cdot y_t + noise$

$y_{t+1} = y_t + noise$

This is a "constant velocity" motion model. We model the velocity ($y$ here) as being constant plus noise. 

In [ ]:
import numpy as np
import adskalman.adskalman as adskalman
import matplotlib.pyplot as plt

In [ ]:
def column(arr):
    """convert 1D array-like to a 2D vertical array

    >>> column((1,2,3))

    array([[1],
           [2],
           [3]])
    """
    arr = np.array(arr)
    assert arr.ndim == 1
    a2 = arr[:, np.newaxis]
    return a2

In [ ]:
# Create a 2-dimensional state space model:
# (x, xvel).
dt = 0.01
true_initial_state = column([0.0, 10.0])
# This is F in wikipedia language.
motion_model = np.array([[1.0, dt],
                         [0.0, 1.0]])

# This is Q in wikipedia language. For a constant velocity form, 
# it must take this specific form to be correct. The
# only free parameter here is `motion_noise_scale`.
motion_noise_scale = 1000.0

# Do not change these values
T3 = dt**3/3
T2 = dt**2/2
motion_noise_covariance = motion_noise_scale*np.array([[T3, T2],
                                           [T2, dt]])

In [ ]:
duration = 0.5
t = np.arange(0.0, duration, dt)

In [ ]:
# Create some fake data with our model.
current_state = true_initial_state
state = []
for _ in t:
    state.append(current_state[:, 0])
    noise_sample = adskalman.rand_mvn(np.zeros(2), motion_noise_covariance, 1).T
    current_state = np.dot(motion_model, current_state) + noise_sample
state = np.array(state)

In [ ]:
fig, axes = plt.subplots(nrows=2)

ax = axes[0]
ax.plot(state[:, 0], '.-', label='true x')
ax.legend()
ax.set_xlabel('t')
ax.set_ylabel('x')

ax = axes[1]
ax.plot(state[:, 1], '.-', label='true x vel')
ax.legend()
ax.set_xlabel('t')
ax.set_ylabel('x vel');

In [ ]:
# Create observation model. We can only observe position.
observation_model = np.array([[1.0, 0.0]])
observation_noise_covariance = np.array([[0.01]])

In [ ]:
# Create noisy observations.
observation = []
for current_state in state:
    noise_sample = adskalman.rand_mvn(np.zeros(1), observation_noise_covariance, 1).T
    current_observation = np.dot(observation_model, column(current_state)) + noise_sample
    observation.append(current_observation[:, 0])
observation = np.array(observation)

In [ ]:
plt.plot(observation[:, 0], '.-', label='observation')
plt.legend()
plt.xlabel('t')
plt.ylabel('x');

In [ ]:
# Run kalman filter on the noisy observations.
y = observation
F = motion_model
H = observation_model
Q = motion_noise_covariance
R = observation_noise_covariance
initx = true_initial_state[:, 0]
initV = 0.1*np.eye(2)

In [ ]:
kfilt = adskalman.KalmanFilter(F, H, Q, R, initx, initV)
xfilt = []
Vfilt = []
for i, y_i in enumerate(y):
    is_initial = i == 0
    xfilt_i, Vfilt_i = kfilt.step(y=y_i, isinitial=is_initial)
    xfilt.append(xfilt_i)
    Vfilt.append(Vfilt_i)
xfilt = np.array(xfilt)
Vfilt = np.array(Vfilt)

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10,8))

ax = axs[0]
t = np.arange(len(xfilt[:, 0]))
low = xfilt[:, 0]-np.sqrt(Vfilt[:, 0, 0])
high = xfilt[:, 0]+np.sqrt(Vfilt[:, 0, 0])
ax.fill_between(t, low, high, alpha=0.2, color='green')

ax.plot(t,state[:, 0], '.-', label='true')
ax.plot(t,observation[:, 0], '.-', label='observed')
ax.plot(t,xfilt[:, 0], '.-', color='green', label='KF estimate')

ax.set_xlabel('t')
ax.set_ylabel('x')
ax.legend()

ax = axs[1]
ax.plot(Vfilt[:, 0, 0], '.-', label='variance')
ax.set_xlabel('t')
ax.set_ylabel('$\sigma^2$')
ax.legend();

In [ ]:
# Now run again with missing data
y[25:35, :] = np.nan
kfilt = adskalman.KalmanFilter(F, H, Q, R, initx, initV)
xfilt = []
Vfilt = []
for i, y_i in enumerate(y):
    is_initial = i == 0
    xfilt_i, Vfilt_i = kfilt.step(y=y_i, isinitial=is_initial)
    xfilt.append(xfilt_i)
    Vfilt.append(Vfilt_i)
xfilt = np.array(xfilt)
Vfilt = np.array(Vfilt)

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10,8))

ax = axs[0]
t = np.arange(len(xfilt[:, 0]))
low = xfilt[:, 0]-np.sqrt(Vfilt[:, 0, 0])
high = xfilt[:, 0]+np.sqrt(Vfilt[:, 0, 0])
ax.fill_between(t, low, high, alpha=0.2, color='green')

ax.plot(t,state[:, 0], '.-', label='true')
ax.plot(t,observation[:, 0], '.-', label='observed')
ax.plot(t,xfilt[:, 0], '.-', color='green', label='KF estimate')

ax.set_xlabel('t')
ax.set_ylabel('x')
ax.legend()

ax = axs[1]
ax.plot(Vfilt[:, 0, 0], '.-', label='variance')
ax.set_xlabel('t')
ax.set_ylabel('$\sigma^2$')
ax.legend();

### Q1 Compared to our 1 dimensional state, we now do a much better job estimating the position, even when we have no observations. Why is that?

(You answer should be a text explanation.)

YOUR ANSWER HERE

In addition to our better ability to estimate position, there is something else pretty interesting going on here. Although our observations are only positions, our estimates also include velocities. Given that the Kalman filter is the Bayesian optimal solution of a linear dynamic system with normally distributed noise, these velocity estimates are the best that can be done according to Bayes' theorem.

Let's look at our velocity estimates:

In [ ]:
fig, axs = plt.subplots(nrows=1, figsize=(10,4))

ax = axs
t = np.arange(len(xfilt[:, 1]))
low = xfilt[:, 1]-np.sqrt(Vfilt[:, 1, 0])
high = xfilt[:, 1]+np.sqrt(Vfilt[:, 1, 0])
ax.fill_between(t, low, high, alpha=0.2, color='green')

ax.plot(t,state[:, 1], '.-', label='true')
ax.plot(t,xfilt[:, 1], '.-', color='green', label='KD estimate')

ax.set_xlabel('t')
ax.set_ylabel('x vel')
ax.legend();

Often we measuring position and then calculating velocity, we take the naive approach and simply measure the difference between adjacent time points and scale by the time interval. What would that look like?

In [ ]:
dy = observation[1:, 0] - observation[:-1, 0]
naive = dy/dt

In [ ]:
fig, axs = plt.subplots(nrows=1, figsize=(10,4))

ax = axs
t = np.arange(len(xfilt[:, 1]))
low = xfilt[:, 1]-np.sqrt(Vfilt[:, 1, 0])
high = xfilt[:, 1]+np.sqrt(Vfilt[:, 1, 0])
ax.fill_between(t, low, high, alpha=0.2, color='green')

ax.plot(t,state[:, 1], '.-', label='true')
ax.plot(t,xfilt[:, 1], '.-', color='green', label='KF estimate')
ax.plot(t[1:],naive, '.-', color='red', label='naive')

ax.set_xlabel('t')
ax.set_ylabel('x vel')
ax.legend();

So, our estimate from the Kalman filter is not only much closer to the true value, it also has the advantage that we have some estimate even for the periods in which we have no observations.